#### Imports

In [47]:
import numpy as np
import pandas as pd
import joblib

In [48]:
import utils as u
from model_utils import Model

In [49]:
%load_ext autoreload
%autoreload 2
###
%load_ext lab_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


#### Text Loader

In [3]:
data_location  = 'interim/'

In [4]:
data_1 = pd.read_excel('interim/data_1.xlsx', sheet_name=None)

In [5]:
data_1_final = []
for sheet in data_1.keys():
    title = data_1[sheet].columns
    title = title[-1]
    temp_df = data_1[sheet].copy()
    temp_df['Source'] = title
    temp_df['Data_1'] = temp_df[title]
    temp_df = temp_df[['Data_1']]
    data_1_final.append(temp_df)

data_1_final  = pd.concat(data_1_final, ignore_index=True)

In [6]:
data_2 = pd.read_excel('interim/data_2.xlsx', sheet_name=None)

In [7]:
data_2_final = []
for sheet in data_2.keys():
    title = data_2[sheet].columns
    title = title[-1]
    temp_df = data_2[sheet].copy()
    temp_df['Source'] = title
    temp_df['Data_2'] = temp_df[title]
    temp_df = temp_df[['Data_2']]
    data_2_final.append(temp_df)

data_2_final  = pd.concat(data_2_final, ignore_index=True)

#### Combining the datasets

In [74]:
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[..., i] = a
    return arr.reshape(-1, la)


def cartesian_product_multi(*dfs):
    idx = cartesian_product(*[np.ogrid[: len(df)] for df in dfs])
    return pd.DataFrame(
        np.column_stack([df.values[idx[:, i]] for i, df in enumerate(dfs)])
    )


test_df = cartesian_product_multi(*[data_1_final, data_2_final])

In [75]:
test_df.shape

(54696668, 2)

In [76]:
test_df.head()

,0,1
0,Yardi,OTTO HARRASSOWITZ GMBH & CO
1,Yardi,GOLDMAN SACHS ASSET MANAGEMENT INTERNATIONAL
2,Yardi,ADDISON LEE PLC DD
3,Yardi,STATE STREET GLOBAL
4,Yardi,SLOANE ROBINSON INVESTMENT SERVICES


In [77]:
test_df.columns = ["data_1", "data_2"]

#### ML Results

In [28]:
# These are words to
words_to_ignore = [
    "capital",
    "management",
    "partners",
    "group",
    "&",
    "asset",
    "investment",
    "of",
    "fund",
    "services",
    "insurance",
    "global",
    "financial",
    "the",
    "investments",
    "consulting",
    "bank",
    "international",
    "solutions",
    "wealth",
    "pension",
    "associates",
    "media",
    "new",
    "london",
    "risk",
    "securities",
    "real",
    "gaming",
    "estate",
    "trust",
    "co",
    "office",
    "family",
    "company",
    "de",
    "research",
    "funds",
    "foundation",
]

# These common words will be used in searching for acronyms
common_words = []
with open("1-1000.txt") as f:
    for line in f:
        common_words.append(line.rstrip())

In [42]:
non_numerical_cols = ["name_a", "name_b", "acr_a", "acr_b"]
feature_columns = [
    "acr_match",
    "JW_distance",
    "LV_distance",
    "num_words_a",
    "num_words_b",
    "len_a",
    "len_b",
]

In [97]:
def predict_comparison(dataframe, name_column_1, name_column_2, trained_model):
    base_df = pd.DataFrame()

    base_df["name_a"] = dataframe[name_column_1].astype(str)
    base_df["name_b"] = dataframe[name_column_2].astype(str)

    # Get the acronyms for a and b
    base_df["acr_a"] = base_df["name_a"].apply(
        lambda x: u.get_acronym(x, words_to_ignore, common_words)
    )
    base_df["acr_b"] = base_df["name_b"].apply(
        lambda x: u.get_acronym(x, words_to_ignore, common_words)
    )

    # Create a numerical field for the same acronyms
    base_df["acr_match"] = base_df.apply(
        lambda row: u.acronym_checker(row["acr_a"], row["acr_b"]), axis=1
    )

    # Get the number of words
    base_df["num_words_a"] = base_df["name_a"].apply(lambda x: u.get_number_words(x))
    base_df["num_words_b"] = base_df["name_b"].apply(lambda x: u.get_number_words(x))

    # Get the length of the strings
    base_df["len_a"] = base_df["name_a"].apply(lambda x: len(str(x)))
    base_df["len_b"] = base_df["name_b"].apply(lambda x: len(str(x)))

    # Get Jaro Winkler distance
    base_df["JW_distance"] = base_df.apply(
        lambda row: u.jaro_winkler_distance(row["name_a"], row["name_b"]), axis=1
    )

    # Get Levenshtein distance
    base_df["LV_distance"] = base_df.apply(
        lambda row: u.levenshtein_distance(row["name_a"], row["name_b"]), axis=1
    )

    # Get the target

    X_test = base_df[feature_columns]

    base_df["prediction"] = trained_model.predict(X_test)
    base_df["prediction_proba"] = trained_model.predict_proba(X_test)[:, 1]

    return base_df

In [21]:
def rolling(df, window, step):
    count = 0
    df_length = len(df)
    while count < (df_length - window):
        yield count, df[count : window + count]
        count += step
    else:
        yield count, df[count : window + count]

### Load Model

In [53]:
# load the model from disk
loaded_model = joblib.load("trained_model/finalized_model.sav")

### Test Model Output

In [84]:
test_df[45400:46000]

,data_1,data_2
45400,Yardi,BOTTICELLI CAPITAL PARTNERS
45401,Yardi,LNR PARTNERS GERMANY
45402,Yardi,NORTHEAST RONGHUI SECURITIES ASSET MANAGEMENT
45403,Yardi,SKADDEN ARPS SLATE MEAGHER & FLOM
45404,Yardi,DECHERT LLP SINGAPORE
...,...,...
45995,CBRE Capital Advisors,BOOKMAKER
45996,CBRE Capital Advisors,GENERALI
45997,CBRE Capital Advisors,TITAN OPPORTUNITIES FUND
45998,CBRE Capital Advisors,GOTTEX FUND MANAGEMENT BOSTON


In [ ]:
matching_df = []
for offset, window in rolling(test_df, 10000, 10000):
    # |     |                      |     |
    # |     The current chunk.     |     How many rows to step at a time.
    # The current offset index.    How many rows in each chunk.
    # your code here
    temp_df = predict_comparison(window, "data_1", "data_2", loaded_model)
    temp_df = temp_df[temp_df["prediction"] == 0]

    
    if not temp_df.empty:
        temp_df.to_csv("calculated/matching_df__{}.csv".format(offset))
    

    pass

In [ ]:
matching_df = pd.concat(matching_df, ignore_index=False)

In [ ]:
matching_df.to_csv("calculated/matching_df.csv")

#### Alternative Version